# Gibbs Sampler MCAR

Starting off...


In [1]:
import numpy as np
from scipy import stats
from matplotlib import pyplot as plt
# Plotting matplot lib in jupyter
%matplotlib inline

# Having all Jupyter display all output buffer rather than most recent one
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
beta = 0.05
seed = 0
filename = 'riboflavinV10.csv'

In [3]:
class Gibbs:
    def __init__(self, filename, beta=0.05, seed=0):
        """

        :param filename: The name/path of the file from which to read data
        :param beta: the probability of missing values
        data_obs = the data matrix with nans at missing value points
        data_mis = the data matrix with nans at observed value points
        data = the data with all true values
        """
        self.seed = seed
        self.beta = beta
        self.data_true, self.data_obs, self.data_mis, self.r = self.init_data(filename)
        self.cov_true, self.corr_true = self.calc_cov_corr(self.data_true)
        self.cov_obs, self.corr_obs = self.calc_cov_corr(self.data_obs)
        self.z_obs = self.calc_z_obs(self.data_obs)
        

# TODO: Implement Gibbs sampler lmao

    def init_data(self, filename):
        """
        Initalizes data: reads data in, and generates a missing matrix with prop beta and creates a matrix of all
        observations without missing data

        :param filename: the name of the file which from to read in data
        """

        # reading in data
        data_true = np.genfromtxt(filename, delimiter=',', skip_header=1)
        if np.all(np.isnan(data_true[:,0])):
            data_true = np.delete(data_true, 0, axis=1)  # gets rid of the first column if nan, aka has row titles
        data_obs = np.full_like(data_true, np.nan)
        data_mis = data_obs.copy()

        # generating the missing entries matrix
        np.random.seed(self.seed)
        observed_matrix = np.random.binomial(1, 1 - self.beta, size=data_true.shape)  # rij = 1 means value is observed

        # nan'ing entries in data corresponding to missing matrix
        for ij, r in np.ndenumerate(observed_matrix):
            if r:
                data_obs[ij] = data_true[ij]
            else:
                data_mis[ij] = data_true[ij]

        # # removing any observations which have nan in their samples
        # idx = [i for i, v in enumerate(data_obs) if np.any(np.isnan(v))]
        # data_obs_nan_rows_removed = np.delete(data_obs, idx, 0) # gets rid of all rows which have nan in them
        return data_true, data_obs, data_mis, observed_matrix

    def calc_cov_corr(self, data_matrix):
        if not np.any(np.isnan(data_matrix)):
            cov = np.cov(data_matrix, rowvar=False)
        else:
            cov = np.full((len(data_matrix[0]), len(data_matrix[0])), np.nan)
        corr = self.calc_corr(data_matrix)
        return cov, corr

    def calc_corr(self, data_matrix):
        corr = np.zeros_like(data_matrix)
        for j in range(len(data_matrix[0])):
            for k in range(len(data_matrix[0])):
                corr[j, k] = stats.kendalltau(data_matrix[:, j], data_matrix[:, k], nan_policy='omit')[0]
        return corr

#     def Gibbs_step_1(self, z_obs, corr_obs):
#         c = corr_obs

#         for j, col in enumerate(z_obs.T):
#             nj = list(range(len(c.T)))
#             del nj[j]

#             c_nj_nj = c[np.ix_(nj, nj)]
#             c_j_nj = c[np.ix_([j], nj)]
#             v = np.matmul(c_j_nj, np.linalg.inv(c_nj_nj)).T  # v_T = C[j,-j]*((C[-j,-j])^-1)
#             sig_sqd = c[j, j] - np.matmul(v.T, c[np.ix_(nj, [j])])

    def calc_z_obs(self, data_obs):
        # setting up the empirical cumulative distribution function
        z_obs = np.full_like(data_obs, np.nan)
        for j in range(len(data_obs[0])):
            for i, row in enumerate(data_obs):
                top = np.sum( ~np.isnan(row) * (row < row[j]) )  # top = sum(rdj * indicator(ydj < yij))
                bot = sum(~np.isnan(row)) + 1  # bot = number_of_obs_x's + 1
                if not np.isnan(data_obs[i, j]):
                    z_obs[i, j] = stats.norm.ppf(top / bot)  # z_obs[ij] = NormalCDF_inv( top / bot)
        return z_obs

## Initalizing Data

In [96]:
data = np.arange(7*4).reshape(7,4)
data = data.astype(np.float64)
data[1,1] = np.nan
data[1,3] = np.nan
c = np.arange(3*3).reshape(3,3)
np.random.seed(4)
r = np.random.binomial(1, 1 - beta, size=(3,3))
data
c
r


array([[ 0.,  1.,  2.,  3.],
       [ 4., nan,  6., nan],
       [ 8.,  9., 10., 11.],
       [12., 13., 14., 15.],
       [16., 17., 18., 19.],
       [20., 21., 22., 23.],
       [24., 25., 26., 27.]])

array([[0, 1, 2],
       [3, 4, 5],
       [6, 7, 8]])

array([[0, 1, 0],
       [1, 1, 1],
       [0, 1, 1]])

In [104]:
# for j in range(len(c.T)):
#     for i in np.where(r.T[j]==0):
# #         c[np.ix_([j],i)]
#           i

where = np.where(np.isnan(data[1]))
np.delete(data[1],where)

array([4., 6.])

In [6]:
g = Gibbs('riboflavinV10.csv', beta=0.1)

/Users/SeanKulinski/anaconda3/envs/py37/lib/python3.7/site-packages/ipykernel_launcher.py:84: RuntimeWarning: invalid value encountered in less


### Gibbs Step 1

In [7]:
# c = g.corr_obs
# r = g.r

In [41]:


for j, col in enumerate(c.T):
    nj = list(range(len(c.T)))
    del nj[j]
    

    c_nj_nj = c[np.ix_(nj, nj)]
    c_j_nj = c[np.ix_([j], nj)]
    v = np.matmul(c_j_nj,np.linalg.inv(c_nj_nj)).T  # v_T = C[j,-j]*((C[-j,-j])^-1)
    sig_sqd = c[j,j] - np.matmul(v.T,c[np.ix_(nj,[j])])
    
#     for i in np.where(r.T[j]==0):
    for i in r.T[j]:
        if not r[i,j]:
            mu = np.matmul(z_obs[np.ix_([i],nj)],v)
        
        

NameError: name 'z_obs' is not defined

In [22]:
c

8

In [28]:
r

1

In [38]:
for i, c, r in enumerate(zip(c,r)):
    print(r)

ValueError: not enough values to unpack (expected 3, got 2)

In [40]:
for i in range(10):
    np.random.seed(0)
    np.random.normal()

1.764052345967664

1.764052345967664

1.764052345967664

1.764052345967664

1.764052345967664

1.764052345967664

1.764052345967664

1.764052345967664

1.764052345967664

1.764052345967664